# Khai báo các thư viện cần sử dụng.

In [ ]:
!pip install keras numpy
!pip install tensorflow


In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import string
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from google.colab import drive

In [ ]:
# mount drive
drive.mount('/content/drive')


# Load dữ liệu EMNIST.

In [ ]:
# Load EMNIST dataset

# Đọc dữ liệu từ các tập tin CSV
file_csv_path = ('/content/drive/My Drive/Colab Notebooks/XuLyAnh/BTL/Handwritten_Data.csv')  # Đường dẫn đến tập tin CSV huấn luyện

# Đọc dữ liệu từ CSV
data = pd.read_csv(file_csv_path)

X = data.iloc[:, 1:].values  # Các cột còn lại là các giá trị pixel
y = data.iloc[:, 0].values  # Cột đầu tiên là nhãn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)

# Đảm bảo rằng dữ liệu có kích thước chính xác (28x28)
X_train = X_train.reshape(-1, 28, 28)  # Định hình lại thành các hình ảnh 28x28
X_test = X_test.reshape(-1, 28, 28)  # Định hình lại thành các hình ảnh 28x28

print(X_train.shape)
print(X_test.shape)


In [ ]:
X_val, y_val = X_train[200000:260000,:], y_train[200000:260000]
X_train, y_train = X_train[:370000,:], y_train[:370000]

print(X_train.shape)

# Reshape lại dữ liệu cho đúng kích thước mà keras yêu cầu.

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

print ("Shape of X_train: {}".format(X_train.shape))
print ("Shape of y_train: {}".format(y_train.shape))

print ("Shape of X_test: {}".format(X_test.shape))
print ("Shape of y_test: {}".format(y_test.shape))

In [ ]:
example = X_train[50]
plt.imshow(example.reshape(28, 28), cmap="gray")
plt.show()

# Áp dụng thuật toán One hot encoding label (Y).

In [ ]:
lett_classes = 27
Y_train = to_categorical(y_train, lett_classes)
Y_val = to_categorical(y_val, lett_classes)
Y_test = to_categorical(y_test, lett_classes)

print('Dữ liệu y ban đầu ', y_train[0])
print('Dữ liệu y sau one-hot encoding ',Y_train[0])

# Định nghĩa model.

In [ ]:
model = Sequential()

# Thêm Convolutional layer với 32 kernel, kích thước kernel 3*3.
model.add(Conv2D(32, (3, 3), activation='sigmoid', input_shape=(28,28,1)))

# Thêm Convolutional layer.
model.add(Conv2D(32, (3, 3), activation='sigmoid'))

# Thêm Max pooling layer.
model.add(MaxPooling2D(pool_size=(2,2)))

# Flatten layer chuyển từ tensor sang vector.
model.add(Flatten())

# Thêm Fully Connected layer với 128 nodes và dùng hàm sigmoid.
model.add(Dense(128, activation='sigmoid'))

# Output layer với 10 node và dùng softmax function để chuyển sang xác suất.
model.add(Dense(27, activation='softmax'))

# Compile model.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Thực hiện train model với data.

In [ ]:
H = model.fit(X_train, Y_train, validation_data=(X_val, Y_val),
batch_size=32, epochs=10, verbose=1)

In [ ]:
# 8. Vẽ đồ thị loss, accuracy của traning set và validation set
fig = plt.figure()
numOfEpoch = 10
plt.plot(np.arange(0, numOfEpoch), H.history['loss'], label='training loss')
plt.plot(np.arange(0, numOfEpoch), H.history['val_loss'], label='validation loss')
plt.plot(np.arange(0, numOfEpoch), H.history['accuracy'], label='accuracy')
plt.plot(np.arange(0, numOfEpoch), H.history['val_accuracy'], label='validation accuracy')
plt.title('Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss|Accuracy')
plt.legend()

# Đánh giá model với dữ liệu test

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

# Lưu model.

In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/XuLyAnh/BTL/mymodel.tflearn')

# Dự đoán kết quả với tệp Test của EMNIST.

In [ ]:
example = X_train[50]

prediction = model.predict(example.reshape(1, 28, 28, 1))

print("\n\nFinal Output: {}".format(np.argmax(prediction)))
decoder_label([np.argmax(prediction)])
plt.imshow(example.reshape(28, 28), cmap="gray")
plt.axis('off')
plt.show()

# Gọi lại model đã train

In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/XuLyAnh/BTL/mymodel.tflearn')

#Giải mã

In [ ]:
def decoder_label(arr):
  alphabet = list(string.ascii_lowercase)  # ['a', 'b', 'c', ..., 'z']

  # Tạo dictionary để ánh xạ số thứ tự với chữ cái
  dict_label = {i: alphabet[i] for i in range(26)}

  # Giải mã mảng số thành các chữ cái
  decoded_arr = [dict_label[i] for i in arr]

  # Chuyển mảng chữ cái thành chuỗi
  decoded_string = ''.join(decoded_arr)

  # In kết quả
  print("Kết quả nhận dạng giải mã: ", decoded_string)

# Dự đoán kết quả với tệp Test của EMNIST sau khi gọi lại Model.

In [ ]:
example = X_train[20]

prediction = model.predict(example.reshape(1, 28, 28, 1))
print("\n\nFinal Output: {}".format(np.argmax(prediction)))
decoder_label([np.argmax(prediction)])

plt.imshow(example.reshape(28, 28), cmap="gray")
plt.show()

# Minh họa xử lý ảnh và chuẩn bị số liệu (Not care about it)

In [ ]:
    # Đọc ảnh vào
    img_path = '/content/drive/My Drive/Colab Notebooks/XuLyAnh/BTL/test6.png'
    img = cv2.imread(img_path)
    img1 = cv2.imread(img_path,0)

    # Resize ảnh
    resized_img = cv2.resize(img1, (4250, 3450))

    # Hiển thị ảnh gốc
    print('original_image')
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    # Hiển thị ảnh xám
    print('\n\ngrayscale_image')
    plt.imshow(img1)
    plt.show()

    # Hiển thị ảnh resize
    print('\n\nresized_image')
    plt.imshow(resized_img)
    plt.show()

    # Cắt ảnh
    crop_img = resized_img[1000:2500, 250:4000]

    # Hiển thị ảnh đã cắt
    print('\n\ncrop_image')
    plt.imshow(crop_img)
    plt.show()

    # Gaussian blur, Otsu's threshold
    blur = cv2.GaussianBlur(crop_img, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # xử lý điểm ảnh nhỏ
    cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < 800:
            cv2.drawContours(thresh, [c], -1, (0,0,0), -1)

    # Hiển thị ảnh sau xử lý Gaussian blur
    print('\n\nGaussian blur')
    plt.imshow(blur)
    plt.show()

    # Hiển thị ảnh sau xử lý Otsu's threshold và điểm ảnh nhỏ
    print('\n\nOtsu threshold')
    plt.imshow(thresh)
    plt.show()


# Sắp xếp viền theo trục x.

In [ ]:
def x_cord_contour(contours):
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        return (int(M['m10']/M['m00']))
    else:
        pass

# Hàm cắt ảnh và hiển thị kết quả từ ảnh được chọn.

In [ ]:
def crop_define(path):

    # Đọc và biểu diễn ảnh từ đường dẫn:
    img = cv2.imread(path)
    img1 = cv2.imread(path,0)

    resized_img = cv2.resize(img1, (4250, 3450))

    plt.imshow(img)
    plt.axis('off')
    plt.show()

    # x,y là chỉ số tọa độ trục x và y, w là width, h là height.
    crop_img = resized_img[1000:2500, 250:4000] #[y:y+h, x:x+w]

    reco_letters(crop_img) # Kết quả

# Hàm nhận dạng chữ cái và xuất file.

In [ ]:
def reco_letters(img):

    # Gaussian blur, Otsu's threshold
    blur = cv2.GaussianBlur(img, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # xử lý điểm ảnh nhỏ
    cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < 800:
            cv2.drawContours(thresh, [c], -1, (0,0,0), -1)

    # findContour dùng để cung cấp đường viền từng chữ cái trên ảnh.
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sắp xếp đường viền từ trái sang phải dựa trên hàm sắp xếp đường viền theo trục x.
    contours_left_to_right = sorted(contours, key = x_cord_contour, reverse = False)

    preprocessed_letter = []

    for (i,c)  in enumerate(contours_left_to_right):
        (x, y, w, h) = cv2.boundingRect(c)

        # tạo viền quanh chữ cái trong ảnh.
        cv2.rectangle(img, (x,y), (x+w, y+h), color=(0,0,0), thickness=2)

        # cắt ảnh và xử lý
        digit = thresh[y:y+h, x:x+w]

        # chuyển về size(18,18)
        resized_digit = cv2.resize(digit, (18,18))

        # thêm padding 5 pixel màu đen vào 4 góc để chuyển ảnh về (28,28)
        padded_digit = np.pad(resized_digit, ((5,5),(5,5)), "constant", constant_values=0)

        # lưu trữ vào preprocessed_letter
        preprocessed_letter.append(padded_digit)

    # Biểu diễn ảnh sau khi tìm được viền các chữ.
    print("\n\n\n")
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()

    # Dự đoán kết quả.
    processed_letter = []

    for digit in preprocessed_letter:
        prediction = model.predict(digit.reshape(1, 28, 28, 1))

        processed_letter.append(np.argmax(prediction))

    # In các chữ cái cần nhận dạng: (Kết quả)
    arr = np.array(processed_letter)
    print('Kết quả nhận dạng mã hóa: ',arr)
    decoder_label(arr)


# Test chương trình với một số bằng tốt nghiệp.

In [ ]:
crop_define('/content/drive/My Drive/Colab Notebooks/XuLyAnh/BTL/test5.png')

In [ ]:
crop_define('/content/drive/My Drive/Colab Notebooks/XuLyAnh/BTL/test6.png')

#Link full project

https://drive.google.com/drive/folders/1BDu3baDpv42pB5ISQV-zLcyavcS_q3T3?usp=sharing